## Artificial Intelligence Assignment 3
## No1. Neural Net
## 20132651 Sungjae Lee

In [1]:
## CNN 을 구현하기 위해 pytorch 의 neural net 패키지를 불러옵니다.

import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
## Convolution 층 2개와 Fully Connected 층 3개로 이루어진 Neural Net 을 하나의 클래스로 정의합니다.
## 해당 클래스에는 전방 전파를 수행하기 위한 forward 함수가 정의되며
## 컨볼루션 연산, 완전연결 다층 퍼셉트론 연산, ReLU 함수의 적용, Pooling 연산등의 수행 과정이 나타납니다.

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        ## pytorch 의 nn.Conv2d 는 2차원 Convolution 연산을 하는 함수입니다.
        ## 3개의 변수가 반드시 들어가며, 앞에서부터 입력 개수, 출력 개수, 커널 크기입니다.
        ## 그 외에 stride, padding, bias 등에 대해 조정할 수 있습니다.
        
        ## conv1 은 1개의 이미지 입력을 5 x 5 커널(필터)에 통과시켜 6개의 출력을 만드는 컨볼루션 층입니다.
        ## conv2 는 위에서의 6개 출력을 입력으로 받아, 5 x 5 커널에 통과시켜 16개의 출력을 만드는 컨볼루션 층입니다.
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        
        ## pytorch 의 nn.Linear 는 완전연결 다층 퍼셉트론 연산을 하는 함수입니다.
        ## 필수적으로 in_features 와 out_features 매개변수가 입력되어
        ## N 개의 입력에 대해 M 개의 출력으로 Fully Connected 된 층을 형성합니다.
        
        ## fc1 ~ fc3 는 각각 400 x 120, 120 x 84, 84 x 10 으로 이루어진 층입니다.
        ## 마지막의 10개 출력은 하나의 이미지가 10개 레이블중 어디에 가까운지 출력하기 위함입니다.
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
    def forward(self, x):
        
        ## 전방 전파를 수행하기 위한 함수 forward 를 정의합니다.
        ## 해당 CNN에서 Pooling 작업은 nn.functional.max_pool2d 함수를 이용하여 진행합니다.
        ## 2 x 2 커널에서 최대값을 뽑아내는 Max Pooling 작업입니다.
        
        ## 먼저 입력 x 에 conv 함수와 relu 함수를 적용시킨 다음, Pooling 을 시행합니다.
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        
        ## Pooling 층의 작동에 있어서, 2 x 2 커널을 만드는 것은 단순히 2 값을 주는 것으로도 가능합니다.
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        
        ## view 함수는 pytorch 에서 Tensor의 형태를 변환시키는 함수 입니다.
        ## numpy 의 reshape 와 동일한 기능이라 볼 수 있습니다.
        ## 이 때, -1 값을 주면 뒤의 값에 따라 자동적으로 크기를 계산하여 reshape 하게 됩니다.
        
        ## conv, relu, pool 연산을 마친 x 를 한 줄의 출력으로 변환합니다.
        x = x.view(-1, self.num_flat_features(x))
        
        ## relu 를 사용한 완전연결 다층 퍼셉트론으로 최종 출력까지 진행합니다.
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
    def num_flat_features(self, x):
        # batch 를 제외한 나머지 feature 의 개수를 곱연산한 값을 반환합니다.
        size = x.size()[1:]
        num_features = 1
        for s in size:
            num_features *= s
        return num_features
    
## net 이름으로 Net 클래스를 하나 생성합니다.    
net = Net()

### (1) 화면 출력 확인 및 의미를 서술

In [3]:
## 위에서 정의한 CNN 의 전체 구조를 한 눈에 확인할 수 있습니다.
## conv1 과 conv2 의 컨볼루션층을 통과하며 학습을 진행한 다음,
## fc 층들을 통과하며 최종적으로 10개의 클래스로 분류되도록 만들어져 있습니다.
## 이 때, 여기서는 보이지 않지만 ReLU 함수를 이용하여 Gradient Vanishing 문제를 해결하고
## Max Pooling 을 이용해 feature 를 요약하는 과정도 포함됩니다.
## 위에서는 정의하지 않았지만 stride(보폭)과 bias 가 기본값으로 설정된 것을 확인할 수 있습니다.

print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


### (2) 정의된 컨볼루션 신경망의 구조 설명 ( 위의 AlexNet 그림 참고 )

In [5]:
# net.parameters() 를 사용하여 정의된 신경망의 학습가능한 매개변수들을 확인 가능합니다.

params = list(net.parameters())

### (3) 화면 출력 확인

In [6]:
# 총 10개의 학습층이 존재하며, 각각의 학습 weight 크기를 확인할 수 있습니다.
# 그중에서도 첫 번째 컨볼루션 층에서는 5 x 5 크기의 커널이 6개 존재합니다.

print(len(params))
for i in range(len(params)):
    print(params[i].size())

10
torch.Size([6, 1, 5, 5])
torch.Size([6])
torch.Size([16, 6, 5, 5])
torch.Size([16])
torch.Size([120, 400])
torch.Size([120])
torch.Size([84, 120])
torch.Size([84])
torch.Size([10, 84])
torch.Size([10])


In [7]:
# 임의의 32*32 입력이 주어졌을때를 가정하고 CNN이 정상적으로 작동하는지 확인합니다.
# 참고로 크기가 다른 입력을 받을 때는 입력의 크기를 재조정하거나 신경망을 수정해야 합니다.

input = torch.randn(1, 1, 32, 32)
out = net(input)

### (4) 화면 출력 확인

In [8]:
## 정상적으로 10개의 레이블에 대해 어떤 레이블에 가까운지에 대한 수치가
## 출력으로 나오는 것을 확인할 수 있습니다.

print(out)

tensor([[ 0.0629, -0.0513, -0.0383, -0.0380, -0.0024,  0.1584, -0.0402, -0.0381,
         -0.1104,  0.0336]], grad_fn=<ThAddmmBackward>)


In [9]:
# 오류 역전파를 통해 그레디언트를 구하기 전에, 모든 가중치의 그레디언트 버퍼를 초기화합니다.
# 이 때, zero_grad 함수를 이용하면 가중치를 손쉽게 초기화할 수 있습니다.
# backward 를 이용하여 역전파를 진행합니다.

net.zero_grad()
out.backward(torch.randn(1, 10))

In [10]:
# 손실함수를 정의하고, 임의의 값들에 대해서 오차 결과를 확인합니다.
# nn 패키지는 많이 사용되는 손실함수들을 제공하며, 여기서는 단순하게 MSE Loss Function 을 사용합니다.

output = net(input)
target = torch.randn(10) # 예시를 위해 랜덤한 값을 target 으로 입력합니다.
target = target.view(1, -1) # output 과 동일한 크기로 변형합니다.
criterion = nn.MSELoss() # criterion 은 기준이란 의미로, 목적 함수를 의미하는 듯 합니다.

loss = criterion(output, target)

### (5) 화면 출력 확인

In [12]:
## 위에서 정의한 목적 함수, MSE 를 이용하여 loss 를 계산한 결과를 출력합니다.

print(loss)

tensor(1.1344, grad_fn=<MseLossBackward>)


In [15]:
## 앞에 코드에서 언급한 것과 같이 오류 역전파하기 전, 그레디언트를 초기화해야 합니다.
## backward() 수행 후 어떤 변화가 있는지 확인하고, 초기화의 필요성을 확인해 봅니다.

## zero_grad() 를 이용하여 모든 parameter 의 가중치를 0으로 초기화 합니다. 
net.zero_grad()

### (6) 화면 출력 확인

In [16]:
## 정상적으로 가중치가 0으로 초기화 되었는지 확인하기 위해 출력해 봅니다.
## 이 때, .grad 를 이용하면 확인이 가능합니다.

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
tensor([0., 0., 0., 0., 0., 0.])


In [17]:
## 가중치를 초기화 한 다음, backward 를 이용하여 오류 역전파를 시행합니다.

loss.backward()

### (7) 화면 출력 확인

In [18]:
## backward 를 진행한 다음, 가중치를 출력하여 확인합니다.
## 0으로 초기화 되었던 가중치들이 오류 역전파를 통해 새로운 값을 가지게 되었습니다.

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad after backward
tensor([ 0.0071, -0.0040,  0.0010, -0.0100, -0.0043,  0.0089])


In [23]:
# 스토캐스틱 경사하강볍 : ((미래)가중치 = (현재)가중치 - 학습률 * 그레디언트)
# 해당 공식을 이용하여 가중치를 갱신하는 코드는 다음과 같습니다.

learning_rate = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)
    ## sub_ 메소드를 이용하여 코드를 효율적으로 구현한 점이 눈에 띕니다.
    ## 찾아본 결과 sub_ 은 sub 의 inplace 버전이라고 합니다.

In [25]:
# 하지만 실제로는 torch.optim 에서 구현되어 있는 SDG, Adam, RMSProp 등을 사용합니다.
# 아래는 오류 역전파에서 최적화하는 방법을 보인 예제 코드입니다.

import torch.optim as optim

## optim 의 SGD 를 사용하여 스토캐스틱 경사 하강을 optimizer 로 정의합니다.
optimizer = optim.SGD(net.parameters(), lr = 0.01)

## 아래는 실제 학습이 진행되는 코드입니다.
optimizer.zero_grad() 
output = net(input)
loss = criterion(output, target)
loss.backward()

## 가중치 초기화, 목적함수 정의, 오류 역전파 등의 과정은 동일합니다
## optimizer.step() 을 이용하여 갱신하는 점이 다른 것을 확인할 수 있습니다.
optimizer.step() 